<a href="https://colab.research.google.com/github/AndreaPareti/ML_course_Pavia_23/blob/main/HiDRaNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A few imports that will be used in the following

In [ ]:
import networkx as nx
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Install Pytorch Geometric libraries

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch_cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

Load Dataset: 200 events per energy set, with primary particle energy in [10, 20, 30, 40, 60, 80, 100] GeV.


In [ ]:
!curl https://cernbox.cern.ch/remote.php/dav/public-files/7GrigA2KQ7uDvR4/hits_piplus_10GeV.csv -o hits_piplus_10GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/wPraydaZYzRL18I/hits_piplus_20GeV.csv -o hits_piplus_20GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/Sckke3sSgbJnZPT/hits_piplus_30GeV.csv -o hits_piplus_30GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/eGtiQK5ciKXyUTu/hits_piplus_40GeV.csv -o hits_piplus_40GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/l6o77MgGgdD0tbr/hits_piplus_60GeV.csv -o hits_piplus_60GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/TnevGLZ0Z1GawlT/hits_piplus_80GeV.csv -o hits_piplus_80GeV.csv
!curl https://cernbox.cern.ch/remote.php/dav/public-files/mPuwiJldxiIvibi/hits_piplus_100GeV.csv -o hits_piplus_100GeV.csv

!ls


Load Dataset:


In [ ]:
def print_device_usage(device):
    tot_memory = torch.cuda.get_device_properties(device).total_memory/1024.0**3
    reserved_memory = torch.cuda.memory_reserved(device)/1024.0**3
    allocated_memory = torch.cuda.memory_allocated(device)/1024.0**3
    free_memory = reserved_memory-allocated_memory  # free inside reserved
    print('Total memory in Gb: %.2f'%tot_memory)
    print('Reserved memory in Gb: %.2f'%reserved_memory)
    print('Allocated memory in Gb: %.2f'%allocated_memory)
    print('Free memory in Gb: %.2f'%free_memory)

print(torch.__version__)
print("CUDA GPU:", torch.cuda.is_available())
torch.manual_seed(42)
# check if a GPU is available. Otherwise run on CPU
device = 'cpu'
args_cuda = torch.cuda.is_available()
if args_cuda: device = "cuda:0"
print('device : ',device)
if args_cuda: print_device_usage(device)
torch.cuda.empty_cache()

2.0.1+cu118
CUDA GPU: False
device :  cpu


Define features of each graph and of nodes (hits in the calirimeter)

And load the data as Pandas Dataframe

In [ ]:
input_features = ['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord']
node_features = ['PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'x', 'y', 'z', 'E']

myDF10 = pd.read_csv('hits_piplus_10GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF20 = pd.read_csv('hits_piplus_20GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF30 = pd.read_csv('hits_piplus_30GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF40 = pd.read_csv('hits_piplus_40GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF60 = pd.read_csv('hits_piplus_60GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF80 = pd.read_csv('hits_piplus_80GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])
myDF100 = pd.read_csv('hits_piplus_100GeV.csv', sep="\t", names=['evtID', 'truthE', 'truthPDG','PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'hitsCoord'])




Import libraries for Graph NNs

In [ ]:
import torch_geometric
from torch_geometric.nn import knn_graph, EdgeConv, global_mean_pool
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, GATConv, GATv2Conv
from torch_cluster import knn_graph

Import Data as dataframe, with every row consisting of one event

The last column is a list of size "n_nodes", and each element is another list of 4 feautures: (X,Y,Z) spatial coordinates and the energy deposit in the hit E (each event has a different number of hits in the calorimeter, and hence, different number of nodes)

Some cleanup is required to pass through this dataset to a GNN: first of all, for each event (graph) the graph-level features are extended to each node.


Let's define a function to clear up this dataset:




In [ ]:
from ast import literal_eval
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
def make_dataset(dataset, df, hits_column_name, target_name, input_features, node_features):
    print(df)
    df[hits_column_name] = df[hits_column_name].fillna("[]").apply(lambda x: eval(x))
    target = np.array(df[target_name])
    n_events = df[hits_column_name].shape[0]
    print("Number of events (Graphs): ", n_events)
    for event in range(n_events):
        # If the number of energy deposits is larger than 1000, filter only the 1000 larger ones
        n_nodes = min(np.array(df[hits_column_name][event]).shape[0], 1000)
        x = np.array([   item[0] for item in  np.array(df[hits_column_name][event])   ])
        y = np.array([   item[1] for item in  np.array(df[hits_column_name][event])   ])
        z = np.array([   item[2] for item in  np.array(df[hits_column_name][event])   ])
        E = np.array([   item[3] for item in  np.array(df[hits_column_name][event])   ])
        truthE = np.full(x.shape, df.truthE[event])
        PMTenergyS = np.full(x.shape, df.PMTenergyS[event])
        PMTenergyC = np.full(x.shape, df.PMTenergyC[event])
        PMTenergyComb = np.full(x.shape, df.PMTenergyComb[event])
        SiPMenergyS = np.full(x.shape, df.SiPMenergyS[event])
        SiPMenergyC = np.full(x.shape, df.SiPMenergyC[event])
        SiPMenergyComb = np.full(x.shape, df.SiPMenergyComb[event])
        hit_data = np.stack((PMTenergyS, PMTenergyC, PMTenergyComb, SiPMenergyS, SiPMenergyC, SiPMenergyComb, x, y, z, E), axis=-1)
        eventDF = pd.DataFrame(hit_data, columns=['PMTenergyS', 'PMTenergyC', 'PMTenergyComb', 'SiPMenergyS', 'SiPMenergyC', 'SiPMenergyComb', 'x', 'y', 'z', 'E']).nlargest(n_nodes, 'E')
        d = Data(x = torch.tensor(eventDF.values, dtype=torch.float), y = torch.tensor(truthE[event], dtype=torch.float), num_nodes = n_nodes)
        dataset.append(d)


Now append all graphs to a list

In [ ]:
hit_dataset = []
make_dataset(hit_dataset, myDF10, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF20, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF30, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF40, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF60, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF80, 'hitsCoord', 'truthE', input_features, node_features)
make_dataset(hit_dataset, myDF100, 'hitsCoord', 'truthE', input_features, node_features)



We can now proceed with splitting the whole dataset in a training and a validation sample:

In [ ]:
from sklearn.model_selection import train_test_split
dataset_size = len(hit_dataset)
val_split = 0.2
val_size = int(val_split * dataset_size)
train_size = dataset_size - val_size
hit_dataset_train, hit_dataset_val = train_test_split(hit_dataset, test_size = val_size, train_size = train_size, shuffle=True)

batch_size = 32
hit_loader_train = DataLoader(hit_dataset_train, batch_size=batch_size, shuffle=True)
hit_loader_val = DataLoader(hit_dataset_val, batch_size=batch_size, shuffle=True)
for data in hit_loader_val:
    print(data)
    print(data.batch)
    break


DataBatch(x=[28081, 10], y=[32], num_nodes=28081, batch=[28081], ptr=[33])
tensor([ 0,  0,  0,  ..., 31, 31, 31])


We can now start with the implementation of the Particle-Net like GNN:

Let's start with considering the features of the nodes we have to consider

In [ ]:
features_to_consider = 'PMTenergyS,PMTenergyC,PMTenergyComb,SiPMenergyS,SiPMenergyC,SiPMenergyComb,x,y,z,E'.split(',')


class ParticleNetEdgeNet(nn.Module):
    def __init__(self, in_size, layer_size):
        super(ParticleNetEdgeNet, self).__init__()

        layers = []
        layers.append(nn.Linear(in_size*2, layer_size))           # directed Graph
        #layers.append(nn.Linear(in_size*2, layer_size))           # un-directed Graph
        layers.append(nn.BatchNorm1d(layer_size))
        layers.append(nn.ReLU())

        for i in range(2):
            layers.append(nn.Linear(layer_size, layer_size))
            layers.append(nn.BatchNorm1d(layer_size))
            layers.append(nn.ReLU())

        self.model = nn.Sequential(*layers)


    def forward(self, x):
        return self.model(x)

    def __repr__(self):
        return "{}(nn={})".format(self.__class__.__name__, self.model)



In [ ]:
class ParticleNet(nn.Module):
    def __init__(self, node_feat_size, num_classes=1):
        super(ParticleNet, self).__init__()
        self.node_feat_size = node_feat_size
        self.num_classes = num_classes

        self.k = 10  #8                                            # K  number of connections to closest nodes in KNN cluster neighbor
        self.num_edge_convs = 3                     # number of stacked EdgeConv blocks
        self.kernel_sizes = [64, 128, 256]          # number of units in three linear transformation layers in EdgeConv blocks
        #self.kernel_sizes = [32, 64, 128]          # number of units in three linear transformation layers in EdgeConv blocks
        self.fc_size = 256                          # units  in the first fully-connected layer after EdgeConv blocks
        self.dropout = 0.1
        self.edge_nets = nn.ModuleList()
        self.edge_convs = nn.ModuleList()

        self.kernel_sizes.insert(0, self.node_feat_size)
        self.output_sizes = np.cumsum(self.kernel_sizes)        # inputs also include previous layer output

        # first EdgeConv block
        self.edge_nets.append(ParticleNetEdgeNet(self.node_feat_size, self.kernel_sizes[1]))
        self.edge_convs.append(EdgeConv(self.edge_nets[-1], aggr="mean"))

        # stack other EdgeConv blocks on the first
        for i in range(1, self.num_edge_convs):
            print("self.num_edge_convs", i)
            # add kernel sizes because of skip connections
            self.edge_nets.append( ParticleNetEdgeNet(self.output_sizes[i], self.kernel_sizes[i+1]) )
            self.edge_convs.append(EdgeConv(self.edge_nets[-1], aggr="mean"))

        self.fc1 = nn.Sequential(nn.Linear(self.output_sizes[-1], self.fc_size))
        self.drouput_layer = nn.Dropout(p=self.dropout)
        self.fc2 = nn.Linear(self.fc_size, self.num_classes)



    def forward(self, data):
        x = data.x
        batch = data.batch
        for i in range(self.num_edge_convs):
            #print(x[:, 2:5])
            # node features:  PMTenergyS, PMTenergyC, PMTenergyComb, SiPMenergyS, SiPMenergyC, SiPMenergyComb, hitX, hitY, hitZ, hitE'
            edge_index = ( knn_graph(x[:, 6:9], self.k, batch) if i==0 else knn_graph(x, self.k, batch) )
            #x = ( torch.cat(self.edge_convs[i](x, edge_index), x), dim=1 )
            x = torch.cat( (self.edge_convs[i](x, edge_index), x), dim=1 )  # concatenating with original features i.e. skip connection
        x = global_mean_pool(x, batch)
        x = self.fc1(x)

        return self.fc2(x)





In [ ]:
def gnn_model_summary(model):
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0]
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)


In [ ]:
def train_test_one_epoch(model,jets_loader, device, optimizer, loss_fn, batch_size, history, mode='train'):
    if mode=='train':
        model.train(True)
    else:
        model.train(False)

    #epoch_loss = 0
    #epoch_acc = 0
    #return epoch_loss, epoch_acc
    running_loss, running_correct = 0.,0.
    tot_iter = len(jets_loader)
    t = tqdm.tqdm(enumerate(jets_loader),total=tot_iter)
    for i, data in t:
        inputs = data # For the inputs we are passing the whole Data() object
        labels = data.y.unsqueeze(1)
        inputs = inputs.to(device)
        labels = labels.to(device)
        #print(data)
        #print(type(data))

        optimizer.zero_grad() # Clear gradients
        #outputs = gnn(inputs) # Perform a single forward pass
        outputs = model(inputs) # Perform a single forward pass

        loss = loss_fn(outputs, labels) # Compute the loss
        if mode=='train':
            loss.backward()  # Derive gradients
            optimizer.step() # Update parameters based on gradients.

        running_loss += loss.item() #.item() returns average loss over the batch
        #print(labels, outputs.T, loss)

        del loss

        #_, predicted = torch.max(F.softmax(outputs,dim=1).data, dim=1) # Adding softmax since there is none in the model, and taking the index of the highest probability.
        #running_correct += float(torch.sum(predicted == labels.data))

        #print(_)

    epoch_loss = running_loss / tot_iter
    #epoch_acc = running_correct / (tot_iter*batch_size)
    history.append( running_loss/tot_iter)

    return epoch_loss, outputs



In [ ]:
import torch.optim as optim
import tqdm


gnn = ParticleNet(node_feat_size=len(features_to_consider), num_classes=1)
gnn.to(device)
#loss_fn = torch.nn.MSELoss()                            # Using Mean Squared Error as loss function for regression
loss_fn = torch.nn.HuberLoss()                            # Using Mean Squared Error as loss function for regression

optimizer = optim.Adam(gnn.parameters(), lr = 0.00075)
EPOCHS = 20
loss_train, loss_val = [],[]
acc_train, acc_val = [],[]
gnn_model_summary(gnn)
print(hit_dataset_val[:10])

history_train = []
history_val = []
for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch + 1))

    loss_train_epoch,  y_pred_train = train_test_one_epoch(gnn, hit_loader_train, device, optimizer, loss_fn, batch_size, history_train, mode = 'train')
    loss_val_epoch, y_pred_val = train_test_one_epoch(gnn, hit_loader_val, device, optimizer, loss_fn, batch_size, history_val, mode = 'test')
    print('LOSS train {:.3f}, valid {:.3f}'.format(loss_train_epoch, loss_val_epoch))
    loss_train.append(loss_train_epoch)
    loss_val.append(loss_val_epoch)
    #save model state with best parameters
    torch.save(gnn.state_dict(), 'best-model-parameters.pt')


In [ ]:
best_model_param = torch.load('best-model-parameters.pt')
gnn.load_state_dict(best_model_param)

In [ ]:
pred_energy = []
truth_energy = []
pred_label = []
true_label = []
for i, data in enumerate(hit_loader_val):
    inputs, truth = data, data.y
    outData = data.x.cpu().detach().numpy()
    y_true = data.y.cpu().detach().numpy()

    inputs = inputs.to(device)
    y_pred = gnn(inputs.to(device)).cpu().detach().numpy().squeeze(1)
    truth_energy.append(y_true)
    pred_energy.append(y_pred)


In [ ]:
def split_iterate(df, colname):
    return [x for __, x in df.groupby(colname)]


truth_energy = np.concatenate(truth_energy).ravel()
pred_energy = np.concatenate(pred_energy).ravel()
gnnRecoDF = pd.DataFrame({'TruthEnergy' : truth_energy, 'GNNenergy' : pred_energy})
print(gnnRecoDF)
results_array = split_iterate(gnnRecoDF, 'TruthEnergy')




In [ ]:
import scipy



print(results_array[0])
print(results_array[1])
resolution = []
fit_resolution = []
fit_Epeak = []
energies = []
#energies = [e.TruthEnergy for e in results_array]
for i in range(len(results_array)):
    trueE = int(results_array[i].TruthEnergy.mean())
    recoE = results_array[i].GNNenergy.mean()
    recoE_std = results_array[i].GNNenergy.std()
    resolution.append(recoE_std/trueE)
    energies.append(trueE)
    print("sigma/E: ", recoE_std/trueE)

    bmin = trueE - 4*np.sqrt(trueE)
    bmax = trueE + 4*np.sqrt(trueE)
    myBins = np.linspace(bmin, bmax, 100)
    #_, bins = plt.hist(myBins, bins=100, range=(bmin, bmax))
    counts, bins = np.histogram(results_array[i].GNNenergy, bins=100, range=(bmin, bmax))
    plt.hist(results_array[i].GNNenergy, bins=10, range=(bmin, bmax), density=1)

    mu, sigma = scipy.stats.norm.fit(results_array[i].GNNenergy)
    best_fit_line = scipy.stats.norm.pdf(bins, mu, sigma)
    fit_resolution.append(sigma)
    fit_Epeak.append(mu)

    plt.plot(bins, best_fit_line)

    #counts, bins = np.histogram(results_array[i].GNNenergy)
    #plt.hist(bins[:-1], bins, weights=counts)
    #plt.savefig('RecoE%d.png' %trueE)
    plt.show()
    plt.clf()


plt.plot(energies, resolution)
plt.xlabel('Energy [GeV]')
plt.ylabel('sigma/E')
plt.savefig('test.png')
plt.clf()

print(energies)
energies = np.array(energies)
x = 1/np.sqrt(energies)
print(x)
y = np.array(fit_resolution)/np.array(fit_Epeak)
print(y)
#plt.errorbar(x, y, yerr=np.sqrt(np.array(fit_resolution)/1000))
coef = np.polyfit(x, y, 1)
polyf = np.poly1d(coef)
plt.plot(x, y, 'yo', x, polyf(x))
print("Par0: ", coef[0], "Par1: ", coef[1])
print("Risoluzione: ", coef[0]*100, "/sqrt(E) + ", coef[1]*100)

plt.show()
